In [1]:
import os 

import pandas as pd

DATA_DIR = "../data"
RESULTS_DIR = os.path.join(DATA_DIR, "results")

In [2]:
results_df = pd.concat([pd.read_csv(os.path.join(RESULTS_DIR, f)) for f in os.listdir(RESULTS_DIR) if f[-4:] == ".csv"], ignore_index=True)

In [3]:
grouped_results_df = pd.concat([
    results_df.groupby(["imputer", "Model", "attr_group", "target"])[["AUC", "Accuracy", "Sensitivity (Recall)", "Specificity"]].mean(),
    results_df.groupby(["imputer", "Model", "attr_group", "target"])[["n_positive", "n_total"]].sum()
], axis=1).reset_index().drop("imputer", axis=1)

In [4]:
grouped_results_df["target"] = grouped_results_df["target"].str.lstrip("Dia_")
grouped_results_df = grouped_results_df.query("target == 'HFD'")
grouped_results_df

,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
0,RandomForest,MICE,HFD,0.693398,0.497175,0.586119,0.581228,6829.0,11266.0
1,RandomForest,expert,HFD,0.642240,0.635607,0.824772,0.309211,6829.0,11266.0
2,RandomForest,expert_blood,HFD,0.604568,0.596722,0.853381,0.077879,6829.0,11266.0
3,RandomForest,secondary,HFD,0.707987,0.656708,0.920631,0.099359,5446.0,9591.0
4,XGBoost,MICE,HFD,0.699324,0.724999,1.000000,0.000000,6829.0,11266.0
5,XGBoost,expert,HFD,0.618781,0.724183,0.998778,0.002182,6829.0,11266.0
6,XGBoost,expert_blood,HFD,0.631633,0.724569,0.999062,0.002020,6829.0,11266.0
8,XGBoost,secondary,HFD,0.594919,0.705423,0.990008,0.001010,5446.0,9591.0


In [5]:
best_models_df = grouped_results_df.groupby(["attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])

C:\Users\borut.flis\AppData\Local\Temp\ipykernel_15356\1337778183.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models_df = grouped_results_df.groupby(["attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])


In [6]:
best_models_df

,,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
attr_group,target,,,,,,,,,
MICE,HFD,XGBoost,MICE,HFD,0.699324,0.724999,1.000000,0.000000,6829.0,11266.0
expert,HFD,RandomForest,expert,HFD,0.642240,0.635607,0.824772,0.309211,6829.0,11266.0
expert_blood,HFD,XGBoost,expert_blood,HFD,0.631633,0.724569,0.999062,0.002020,6829.0,11266.0
secondary,HFD,RandomForest,secondary,HFD,0.707987,0.656708,0.920631,0.099359,5446.0,9591.0


In [7]:
rename_cols = {"Sensitivity (Recall)": "Sensitivity", "AUC": "c-index", "target": "Predicting", "attr_group": "Attributes"}
final_selection = ["Attributes", "Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

In [ ]:
compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].iloc[:-1].rename(columns=rename_cols)
best_models_df["Majority Accuracy"] = best_models_df["n_positive"]/ best_models_df["n_total"]

In [9]:
compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].rename(columns=rename_cols)
compare_attr_df["Attributes"] = ["MICE", "Expert Selection", "Expert + Blood", "Secondary"]
compare_attr_df["Majority Accuracy"] = compare_attr_df["n_positive"]/ compare_attr_df["n_total"]
compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")] = compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")].round(3)
#compare_attr_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_attrs.csv"), index=None)
compare_attr_df.loc[:, final_selection]

,,Attributes,Predicting,Model,c-index,Accuracy,Majority Accuracy,Sensitivity,Specificity
attr_group,target,,,,,,,,
MICE,HFD,MICE,HFD,XGBoost,0.699,0.725,0.606,1.000,0.000
expert,HFD,Expert Selection,HFD,RandomForest,0.642,0.636,0.606,0.825,0.309
expert_blood,HFD,Expert + Blood,HFD,XGBoost,0.632,0.725,0.606,0.999,0.002
secondary,HFD,Secondary,HFD,RandomForest,0.708,0.657,0.568,0.921,0.099


In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert_blood", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert_blood.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[[pd.IndexSlice["secondary", "HFD"]], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_secondary.csv"), index=None)
compare_expert_df.loc[:, final_selection]